In [1]:
%load_ext autoreload
%autoreload 2

In [362]:
from wdtools import *

In [32]:
#setID = 'Set001'; nm_to_add=0
#setID = 'Set002'; nm_to_add=1420
#setID = 'Set003'; nm_to_add = 2143
setID = 'Set004'; nm_to_add = 2878

In [33]:
wd = combine_wd_tables(setID=setID, nm_to_add=nm_to_add)

In [59]:
unmapped_notes = pd.read_excel(f'{inpath}\\notes\\LCOGUnmappedRecordsJB.xlsx', skiprows=1)

In [13]:
mappedIDs = ['WD2017-0198', 'WD2017-0229', 'WD2019-0082', 'WD2021-0179']
holdIDs = ['WD2022-0291', 'WD2021-0703']
nomapIDs = ['WD2021-0181', 'WD2022-0158']
maybeIDs = ['WD2017-0050', 'WD2017-0022', 'WD2020-0531']

In [15]:
skipIDs = mappedIDs + holdIDs + nomapIDs + maybeIDs

In [16]:
wdIDs = [wid for wid in unmapped_notes.wetdet_delin_number.unique() if wid not in skipIDs]

In [34]:
len(wdIDs)

31

In [35]:
toCheck_df = wd[wd.wetdet_delin_number.isin(wdIDs)]

In [36]:
review_loop_r1(df=toCheck_df, partial=True, idx=True)

0.0% digitized, 15 records remained, expected to be done in about 12 hours...
WD2021-0031
index = 27
   county     trsqq parcel_id   latitude   longitude  record_ID notes  \
26  Crook  15S14E01   702,800  44.292086 -120.989141       2905         
27  Crook  15S15E07       300  44.292086 -120.989141       2906         
28  Crook  15S14E12   100,401  44.292086 -120.989141       2907         
29  Crook  15S14E06       600  44.292086 -120.989141       2908         

   missinglot  
26          N  
27          N  
28          N  
29          N  
Press 'p' to pause or any key to stop...p
Press 'a' to add the wd record or 'c' to continue...c
6.7% digitized, 14 records remained, expected to be done in about 11 hours...
WD2018-0002
index = 97
        county     trsqq parcel_id   latitude   longitude  record_ID  \
96  Hood River  03S09E00  100p,ROW  45.319481 -121.628633       2975   

           notes missinglot  
96  ROW, Partial          N  
Press 'p' to pause or any key to stop...p
Press 'a'

In [37]:
issuepath = inpath + '\\GIS\\ArcGIS Pro Project\\DataReview\\issueIDs.gdb'

In [38]:
mapped = [lyr for lyr in fiona.listlayers(issuepath) if (lyr != 'WD2017_0229') and ('L' not in lyr)]

In [40]:
len(mapped)

35

In [81]:
gdf = merge_single_partial_file(setID=False, wIDlist=mapped, from_set=False)

In [82]:
gdf = gdf.dissolve('wdID')
gdf['wdID'] = gdf.index
gdf.reset_index(drop=True, inplace=True)

In [83]:
code0 = ["WD2019-0300", "WD2021-0597", "WD2019-0436", "WD2020-0568", "WD2019-0300"]
code1 = ["WD2017-0314", "WD2019-0046", "WD2019-0496", "WD2021-0416", "WD2021-0653"]

In [84]:
gdf.loc[gdf.wdID.isin(code0),'code'] = 0
gdf.loc[gdf.wdID.isin(code1),'code'] = 1
gdf.loc[gdf.code.isnull(),'code'] = 2

In [152]:
gdf.shape

(35, 6)

In [74]:
wd_df = get_all_wd(num=4)

In [76]:
set_dict = dict(zip(wd_df.wetdet_delin_number, wd_df.SetID))

In [85]:
gdf.loc[:,'SetID'] = gdf.wdID.map(set_dict)

In [195]:
sa_gdf_all = get_all_SA(num=4)

In [80]:
sa_gdf_all.head(3)

,code,wdID,geometry,SetID
0,2,WD2017-0003,"POLYGON ((447632.599 1578055.844, 447349.923 1...",1
1,2,WD2017-0014,"POLYGON ((659383.287 224983.480, 659986.729 22...",1
2,0,WD2017-0016,"POLYGON ((266404.216 519932.161, 266504.107 51...",1


In [226]:
all_gdf = pd.concat([sa_gdf_all[['wdID', 'code', 'SetID', 'geometry']],
                    gdf[['wdID', 'code', 'SetID', 'geometry']]], ignore_index=True)

In [88]:
start = time.time()
all_taxlot = combine_taxlot()
end = time.time()
print(f'it took {round((end - start)/60, 0)} minutes to complete')

it took 9.0 minutes to complete


In [89]:
start = time.time()
all_mapIdx = read_all_mapIdx()
end = time.time()
print(f'it took {round((end - start)/60, 0)} minutes to complete')

it took 1.0 minutes to complete


In [201]:
gdf.loc[gdf.wdID=='WD2021-0495', 'wdID'] = 'WD2021-0031'
gdf = gdf[~gdf.wdID.isin(['WD2019-0259', 'WD2018-0352', 'WD2019-0300'])]
sa_gdf_all = sa_gdf_all[~sa_gdf_all.wdID.isin(['WD2019-0259'])]
sa_gdf_all = sa_gdf_all[~((sa_gdf_all.wdID=='WD2021-0686') & (sa_gdf_all.SetID==1))]
sa_gdf_all = sa_gdf_all[~((sa_gdf_all.wdID=='WD2022-0026') & (sa_gdf_all.SetID==2))]

In [229]:
sa_gdf_all[sa_gdf_all.wdID.isin(wdID_list)]

,code,wdID,geometry,SetID
453,0,WD2019-0511,"POLYGON ((667916.072 1236023.094, 667881.278 1...",1
1203,0,WD2019-0511,"MULTIPOLYGON Z (((598409.865 958092.642 0.000,...",2
2183,0,WD2019-0511,"MULTIPOLYGON (((601741.516 983380.405, 601794....",4


In [103]:
wd_df.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'Coord-Source', 'DocumentName', 'DecisionLink', 'is_batch_file',
       'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'record_ID',
       'notes', 'recyear', 'IDyear', 'missinglot', 'SetID'],
      dtype='object')

In [233]:
wd_df.loc[wd_df.county == 'TIllamook', 'county'] = 'Tillamook'

In [234]:
wd_df.loc[wd_df.county == 'Yamhill and Washington', 'county'] = 'Yamhill'

all_gdf = all_gdf.dissolve('wdID')
all_gdf['wdID'] = all_gdf.index
all_gdf.reset_index(drop=True, inplace=True)

In [275]:
selectedID = wd_df.parcel_id.astype(str) != 'nan'
wd_df = wd_df.copy()[selectedID]
wd_df['notes'] = wd_df.copy()['parcel_id'].apply(lambda x: make_notes(x))

In [280]:
wd_dt = wd_df[wd_df.notes == '']

In [363]:
start = time.time()
out = split_SA_by_wid_in_df(wd_df=wd_dt, 
                            sa_gdf_all=all_gdf, 
                            all_mapIdx=all_mapIdx, 
                            all_taxlot=all_taxlot, 
                            em_wids=all_gdf.wdID.unique(),
                            export=True, 
                            outnm='wd_sa_gdf', 
                            review=True)
end = time.time()
print(f'it took {round((end - start)/60, 0)} minutes to complete')

it took 3.0 minutes to complete
